### imports

In [18]:
import lmdb
import numpy as np
import matplotlib.pyplot as plt
import PIL.ImageOps
from PIL import Image
from sklearn.model_selection import StratifiedKFold

import env
from utils import KaggleCameraDataset, progress_iter

%matplotlib inline
%load_ext autoreload
%autoreload 2

ImportError: No module named lmdb

## load training data

In [2]:
train_data = KaggleCameraDataset('../data/', train=True, lazy=True)

## calculate dataset size

In [7]:
crop_size = 1024
n_crops = 1
N = len(train_data)

size = crop_size**2*3*n_crops*N
print size
print "Dataset will occupy {0:.2f} GB".format(size/float(2**30))

8650752000
Dataset will occupy 8.06 GB


In [6]:
X = np.zeros((N, 256, 256, 3), dtype=np.uint8)
print X.nbytes

540672000


In [13]:
np.ceil(np.log2(size))

34.0

In [17]:
int(2.**np.ceil(np.log2(size)))

17179869184

## #1 gen data (center crops only)

In [16]:
X = np.zeros((N, crop_size, crop_size, 3), dtype=np.uint8)

pos = 0
for x, _ in progress_iter(train_data, verbose=True):
    w = x.size[0]
    h = x.size[1]
    x = x.crop((w/2-crop_size/2, h/2-crop_size/2,
                w/2+crop_size/2, h/2+crop_size/2))
    
    X[pos, ...] = np.asarray(x, dtype=np.uint8)
    pos += 1
    
y = np.asarray(train_data.y)

A Jupyter Widget

## #1.1 save to lmbd

## #2 gen aug data (center crops + D4 group)

In [4]:
X = np.zeros((N * 8, crop_size, crop_size, 3), dtype=np.uint8)

pos = 0
for x, _ in progress_iter(train_data, verbose=True):
    w = x.size[0]
    h = x.size[1]
    x = x.crop((w/2-crop_size/2, h/2-crop_size/2,
                w/2+crop_size/2, h/2+crop_size/2))
    
    X[pos, ...] = np.asarray(x, dtype=np.uint8)
    pos += 1
    X[pos, ...] = np.asarray(x.transpose(Image.ROTATE_90), dtype=np.uint8)
    pos += 1
    X[pos, ...] = np.asarray(x.transpose(Image.ROTATE_180), dtype=np.uint8)
    pos += 1
    X[pos, ...] = np.asarray(x.transpose(Image.ROTATE_270), dtype=np.uint8)
    pos += 1
    
    x = PIL.ImageOps.mirror(x)
    
    X[pos, ...] = np.asarray(x, dtype=np.uint8)
    pos += 1
    X[pos, ...] = np.asarray(x.transpose(Image.ROTATE_90), dtype=np.uint8)
    pos += 1
    X[pos, ...] = np.asarray(x.transpose(Image.ROTATE_180), dtype=np.uint8)
    pos += 1
    X[pos, ...] = np.asarray(x.transpose(Image.ROTATE_270), dtype=np.uint8)
    pos += 1

y = np.asarray(train_data.y).repeat(8)

A Jupyter Widget

## split into folds
### what I'm gonna do

In [17]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1337)
z = np.asarray([0, 1]).repeat(5)
for train_ind, test_ind in skf.split(z, z):
    print train_ind, test_ind

new_ind = np.concatenate( [test_ind for _, test_ind in skf.split(z, z)] )
print new_ind

[0 1 2 4 5 6 7 9] [3 8]
[0 1 3 4 5 6 8 9] [2 7]
[0 2 3 4 5 7 8 9] [1 6]
[1 2 3 4 6 7 8 9] [0 5]
[0 1 2 3 5 6 7 8] [4 9]
[3 8 2 7 1 6 0 5 4 9]


In [18]:
new_ind = np.concatenate( [test_ind for _, test_ind in skf.split(y, y)] )
print new_ind.shape

(2750,)


In [19]:
X = X[new_ind]
y = y[new_ind]

### reshape for easy extraction of train, val

In [20]:
X = X.reshape(len(X)/5, 5, 256, 256, 3).transpose((1, 0, 2, 3, 4))
X.shape

(5, 550, 256, 256, 3)

In [21]:
y = y.reshape(len(y)/5, 5).T
y.shape

(5, 550)

### save

In [22]:
np.save('../data/X_folds.npy', X)
np.save('../data/y_folds.npy', y)